In [41]:
import pandas as pd
import os
import numpy as np
import re
from sklearn.utils import shuffle
from collections import OrderedDict
import sys
sys.path.append('../')
from log_parser import Drain


log_file_path = 'data/unstructured/HDFS/'
label_file_name = 'data/unstructured/HDFS/anomaly_label.csv'
unstructured_log_filename = 'HDFS_2k.log'
structured_log_file_path = 'data/structured/HDFS/'
structured_log_filename = 'HDFS_2k.log_structured.csv'
log_templates_file = 'HDFS_2k.log_templates.csv'



def parseLog(log_file_path, log_file_name, structured_log_file_path, log_type):
    if log_type == 'HDFS':
        log_format = '<Date> <Time> <Pid> <Level> <Component>: <Content>'

    # Regular expression list for optional preprocessing (default: [])
    regex      = [
        r'blk_(|-)[0-9]+' , # block id
        r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
        r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
    ]
    st         = 0.5  # Similarity threshold
    depth      = 4  # Depth of all leaf nodes

    parser = Drain.LogParser(log_format, indir=log_file_path, outdir=structured_log_file_path,  depth=depth, st=st, rex=regex)
    parser.parse(log_file_name)

## parse the logs - convert unstructured to structured log
parseLog(log_file_path, unstructured_log_filename, structured_log_file_path, 'HDFS')
    

## read structured log 
#print("Loading", structured_log_file_path+structured_log_filename)
structured_log = pd.read_csv(structured_log_file_path+structured_log_filename, engine='c', na_filter=False, memory_map=True)



Parsing file: data/unstructured/HDFS/HDFS_2k.log
Processed 50.0% of log lines.
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:00.201106]


In [49]:
# # print(struct_log)
# log_temp = pd.read_csv(structured_log_file_path+log_templates_file)
# log_temp.sort_values(by = ["Occurrences"], ascending=False, inplace=True)
# log_temp_dict = {event: idx+1 for idx , event in enumerate(list(log_temp["EventId"])) }
# print(log_temp_dict)
# structured_log["EventId"] = structured_log["EventId"].apply(lambda x: log_temp_dict.get(x, -1))

# Create a map of BlockId vs the EventSequence. 
data_dict = OrderedDict()

for idx, row in structured_log.iterrows():
    block_ids = set(re.findall(r'(blk_-?\d+)', row['Content']))
    for block in block_ids:
        if not block in data_dict:
            data_dict[block] = []
        data_dict[block].append(row['EventId'])
data_df = pd.DataFrame(list(data_dict.items()), columns=['BlockId', 'EventSequence'])



# Add anomaly label to the data tagged to block id
label_data = pd.read_csv(label_file_name, engine='c', na_filter=False, memory_map=True)
label_data = label_data.set_index('BlockId')
label_dict = label_data['Label'].to_dict()
data_df['Label'] = data_df['BlockId'].apply(lambda x: 1 if label_dict[x] == 'Anomaly' else 0)


data_df.to_csv('data_instances.csv', index=False)


In [48]:
def split_data(x_data, y_data=None, train_ratio=0):
    num_train = int(train_ratio * x_data.shape[0])
    x_train = x_data[0:num_train]
    x_test = x_data[num_train:]
    y_train = y_data[0:num_train]
    y_test = y_data[num_train:]
    return (x_train,y_train),(x_test,y_test)

# Split train and test data
(x_train, y_train), (x_test, y_test) = split_data(data_df['EventSequence'].values, 
     data_df['Label'].values,0.5)

print(y_train.sum(), y_test.sum())



# (x_train, y_train), (x_test, y_test) = dataloader.load_HDFS(...)




## apply ML to it




41 28
